In [4]:
from datasets import Dataset

subset_ds = Dataset.load_from_disk("data/subset_openmathinstruct_1/256K")

FileNotFoundError: No such files: '/home/quang_ai/AdvancedLLMReasoning/data_utils/data/subset_openmathinstruct_1/256K/dataset_info.json', nor '/home/quang_ai/AdvancedLLMReasoning/data_utils/data/subset_openmathinstruct_1/256K/state.json' found. Expected to load a `Dataset` object but provided path is not a `Dataset`.

In [1]:
system_msg = (
    "Solve the problem step by step. You can use Python code if needed.\n"
    "If you write code, use python markdown blocks (```python ... ```).\n"
    "Output ONLY the final number inside \\boxed{}. Example: \\boxed{42}."
)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

def format_llama3_prompt(example):
    # 1. Tách phần Instruction (Luật) để làm System Prompt
    system_msg = (
        "Solve the problem step by step. You can use Python code if needed.\n"
        "If you write code, use python markdown blocks (```python ... ```).\n"
        "Output ONLY the final number inside \\boxed{}. Example: \\boxed{42}."
    )
    
    # 2. Tạo danh sách messages
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['solution']} # Đây là phần target để train
    ]
    
    # 3. Áp dụng template
    # tokenize=False để trả về chuỗi text, giúp bạn kiểm tra được định dạng trước khi token hóa
    return tokenizer.apply_chat_template(messages, tokenize=False)

# Ví dụ test thử
sample_data = {
    "question": "Calculate 12 + 8.",
    "solution": "```python\nprint(12+8)\n```\nThe answer is \\boxed{20}."
}

print(format_llama3_prompt(sample_data))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Dec 2025

Solve the problem step by step. You can use Python code if needed.
If you write code, use python markdown blocks (```python ... ```).
Output ONLY the final number inside \boxed{}. Example: \boxed{42}.<|eot_id|><|start_header_id|>user<|end_header_id|>

Calculate 12 + 8.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```python
print(12+8)
```
The answer is \boxed{20}.<|eot_id|>
